In [18]:
### run once to install nltk and download stopwords
# !pip3 install nltk
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/christinabrady/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:
import numpy as np
import pandas as pd
import datetime
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams, bigrams, trigrams

In [10]:
reviews = pd.read_json("processed_data/yelp_team7_dataset_review.json")
pandemic_reviews = reviews[reviews.date >= datetime.datetime(2020, 2, 1)] 
pandemic_reviews.shape
# stop_words = set(stopwords.words('english'))

(169762, 9)

### Number of Characters per Review (includes punctuation)

In [ ]:
review_chars = [len(list(x)) for x in reviews.text]

In [ ]:
review_chars_fig =px.histogram(review_chars,
                          title = "Number of characters in reviews",
                         labels = {'value': 'Number of characters'})
review_chars_fig.show()

In [ ]:
reviews["text_char_len"] = review_chars
review_len_x_star_fig = px.box(reviews, 
                                  x = "stars", 
                                  y = "text_char_len", 
                                  title = "Length of Review by Stars")
review_len_x_star_fig.show()

### Number of words per review

In [7]:
## pull review text into its own list to make the code easier to read
review_text = reviews.text.tolist()
## before taking out ALL punctuation, let's take out apostrophes and leave the contraction combined
review_text = [re.sub("'", "", review).strip() for review in review_text]
review_text_no_punc = [re.sub("[^a-zA-Z]", " ", review).strip() for review in review_text]

In [13]:
### Let's see if that worked
review_text_no_punc[np.random.randint(0, len(review_text_no_punc))]

'This review is more about the ambiance and drinks as thats all I had the chance to experience  Im in this area all the time and never noticed this place bc its not open until the afternoon evening I believe    Met a friend here for drinks and I love the laid back feel  industrial lighting and sleek interior  There werent too many people there so it was pretty quiet  The wait staff was a little slow initially taking my drink order while I waited on my party  but once she finally came over she was very attentive the rest of the evening    I had a glass of Chardonnay and my friend had a cocktail  She also ordered the Brussel sprouts which I tasted and they were delicious  I consider myself a sprout connoisseur so I give them a   out of       I will be back for sure to have the pizza and taste the wings'

In [ ]:
words_per_review = [len(text.split()) for text in review_text_no_punc]
words_per_review_fig =px.histogram(words_per_review,
                          title = "Number of words in reviews",
                         labels = {'value': 'Number of characters'})
words_per_review_fig.show()

In [21]:
### What are the most common words overall?
# tokens = [word_tokenize(words) for words in review_text_no_punc]
tokens = [word.split() for words in review_text_no_punc for word in words]

KeyboardInterrupt: 

We want to keep negatives as they probably have meaning for us.